In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Input,MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD,Nadam,AdamW,RMSprop


import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

In [3]:

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# ---- AUTOMOBILE vs NON-AUTOMOBILE ----
# Prepare balanced dataset for automobile (class 1) vs non-automobile
def prepare_binary_data(x, y, target_class):
    pos_idx = np.where(y == target_class)[0]
    neg_idx = np.where((y != target_class))[0]
    n = min(len(pos_idx), len(neg_idx))
    idx = np.concatenate([pos_idx[:n], np.random.choice(neg_idx, n, replace=False)])
    np.random.shuffle(idx)
    return x[idx].astype('float32') / 255.0, (y[idx] == target_class).astype(int)

x_train_auto, y_train_auto = prepare_binary_data(x_train, y_train, target_class=1)
x_test_auto, y_test_auto = prepare_binary_data(x_test, y_test, target_class=1)



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [4]:
model = tf.keras.models.load_model('ship_non_ship_cnn_model.keras')


for layer in model.layers:
    layer.trainable = False

# ---- Unfreeze only the last Conv2D and GlobalAveragePooling2D layers ----
# Find last Conv2D
last_conv2d = None
for layer in reversed(model.layers):
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv2d = layer
        break

# Find GlobalAveragePooling2D
gap_layer = None
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.GlobalAveragePooling2D):
        gap_layer = layer
        break

if last_conv2d is not None:
    last_conv2d.trainable = True
if gap_layer is not None:
    gap_layer.trainable = True

# ---- Recompile ----
model.compile(optimizer=Adam(3.37e-3), loss='binary_crossentropy', metrics=['accuracy'])#

In [5]:
# ---- Fine-tune on automobile vs non-automobile ----


def apply_cutout(image, size=8, n_holes=1):
    h, w = image.shape[0], image.shape[1]
    for n in range(n_holes):
        # Random position of cutout
        y = np.random.randint(h)
        x = np.random.randint(w)

        # Ensure cutout stays within image bounds
        y1 = np.clip(y - size // 2, 0, h)
        y2 = np.clip(y + size // 2, 0, h)
        x1 = np.clip(x - size // 2, 0, w)
        x2 = np.clip(x + size // 2, 0, w)

        # Set the cutout region to zero
        image[y1:y2, x1:x2, :] = 0
    return image
def cutout_preprocess(img):
    return apply_cutout(img)
x_train_auto_cutout = x_train_auto.copy()
for i in range(len(x_train_auto_cutout)):
    x_train_auto_cutout[i] = apply_cutout(x_train_auto_cutout[i])

datagen = ImageDataGenerator(
    preprocessing_function=cutout_preprocess,
    rotation_range=6,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)
datagen.fit(x_train_auto_cutout)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

print("Fine-tuning ship model on automobile vs non-automobile images...")
history_auto = model.fit(
    datagen.flow(x_train_auto_cutout, y_train_auto, batch_size=16),
    epochs=20,
    validation_data=(x_test_auto, y_test_auto),
    callbacks=callbacks,
    steps_per_epoch=len(x_train_auto_cutout) // 16,
    verbose=1
)

# Save final model
model.save('auto_non_auto_cnn_model_from_ship.keras')


Fine-tuning ship model on automobile vs non-automobile images...
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 65ms/step - accuracy: 0.5742 - loss: 0.7376 - val_accuracy: 0.6920 - val_loss: 0.5868 - learning_rate: 0.0034
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 61ms/step - accuracy: 0.6590 - loss: 0.6159 - val_accuracy: 0.6775 - val_loss: 0.5897 - learning_rate: 0.0034
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 38s 61ms/step - accuracy: 0.6619 - loss: 0.6232 - val_accuracy: 0.7220 - val_loss: 0.5639 - learning_rate: 0.0034
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 38s 61ms/step - accuracy: 0.6711 - loss: 0.6157 - val_accuracy: 0.6880 - val_loss: 0.5846 - learning_rate: 0.0034
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.6784 - loss: 0.6152 - val_accuracy: 0.7170 - val_loss: 0.5576 - learning_rate: 0.0034
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 60ms/step - accuracy: 0.6694 - loss: 0.6140 - val_accuracy: 0.7315 - val_loss: 0.5552 - learning_rate: 0.0034
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 63ms/step - accuracy: 0.6678 - loss: 0.6129